In [2]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
key = 'RV2SIKGG44RG7K9C'
ts = TimeSeries(key=key, output_format='json')

/Users/Zjxi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
tickers = [
    'XIC.TO', 'XST.TO', 'ZIN.TO', 'XFN.TO', 
    'XMA.TO', 'XUT.TO', 'XIT.TO', 'XEG.TO',
    'XRE.TO'
    ]
combined_data = pd.DataFrame()

# loop through each tick and fetch the daily data
# outputsize='full': returns the full-length time series 
# of 20+ years of historical data
for ticker in tickers:
    data, metadata = ts.get_daily(symbol=ticker,
                                  outputsize='full')

    # convert JSON to DataFrame
    df = pd.DataFrame.from_dict(data=data, orient='index')
    df.index.name = 'Date'
    df.reset_index(inplace = True)

    # keep only the date and the closing price
    df = df[['Date', '4. close']]
    df.columns = ['Date', ticker]

    # set the 'Date' column as the index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # merge the data into the combined_data
    # how='outer': sql-like outer join, keep all the rows
    combined_data = pd.merge(combined_data, df, 
                             how='outer', left_index=True, 
                             right_index=True)
    

In [6]:
combined_data.to_csv('price.csv', index=True)